In [18]:
import cv2
import numpy as np
import dlib
from imutils import face_utils

In [19]:
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()
while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Cannot read frame.")
        break
    cv2.imshow('Video Capture', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [20]:
face_detector = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor("C:\\Users\\rakul\\Downloads\\archive\\shape_predictor_68_face_landmarks.dat")

In [21]:
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()
while True:
    ret, frame = cap.read()

    if not ret:
        print("Error: Cannot read frame.")
        break

    # Convert the frame to grayscale for faster processing
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the grayscale frame
    faces = face_detector(gray)

    # Iterate over detected faces
    for face in faces:
        # Determine facial landmarks
        landmarks = landmark_predictor(gray, face)
        landmarks = face_utils.shape_to_np(landmarks)

        # Draw the facial landmarks on the frame
        for (x, y) in landmarks:
            cv2.circle(frame,(x,y),1,(255, 255, 255),-1)

    # Display the frame with landmarks
    cv2.imshow('Facial Landmarks', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [22]:
import playsound
import threading
import time

# Function to play alarm sound
def play_alarm_sound():
    playsound.playsound("mixkit-classic-alarm-995.wav")

In [23]:
# Function to compute the Euclidean distance between two points
def compute_distance(pointA, pointB):
    distance = np.linalg.norm(pointA - pointB)
    return distance

# Function to calculate the Eye Aspect Ratio (EAR)
def calculate_ear(eye_points):
    vertical_distance = compute_distance(eye_points[1], eye_points[5]) + compute_distance(eye_points[2], eye_points[4])
    horizontal_distance = compute_distance(eye_points[0], eye_points[3])
    eye_aspect_ratio = vertical_distance / (2.0 * horizontal_distance)
    return eye_aspect_ratio

# Function to determine the blink state based on EAR and normal EAR
def get_blink_state(ear, normal_ear):
    if normal_ear*0.8 <= ear:
        return 2  # Eyes open if 80% of the eye or more is open
    elif ear >= normal_ear * 0.5:
        return 1  # Eyes drowsy
    else:
        return 0  # Eyes closed if less than 50% of eye is open

In [25]:
# Initialize counters and status variables
sleep_counter = 0
drowsy_counter = 0
active_counter = 0
alert_status = ""
status_color = (0, 0, 0)
alarm_play_count = 0
frame_count = 0
normal_ear = 0


# Initialize video capture from the default camera
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Unable to open camera")
    exit()
    
# Capture first 20 frames to calculate the normal EAR
print("Capturing initial frames to calculate normal EAR. Please keep your eyes open.")
time.sleep(2)  # Give the driver some time to prepare

while frame_count < 20:
    ret, frame = cap.read()
    if not ret:
        print("End of video stream")
        break

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    detected_faces = face_detector(gray_frame)

    for face in detected_faces:
        landmarks = landmark_predictor(gray_frame, face)
        landmarks = face_utils.shape_to_np(landmarks)

        left_eye_ear = calculate_ear(landmarks[36:42])
        right_eye_ear = calculate_ear(landmarks[42:48])

        normal_ear += (left_eye_ear + right_eye_ear) / 2.0
        frame_count += 1

normal_ear /= 20.0
print(f"Normal EAR calculated: {normal_ear}")

# Main loop to process video frames
while True:
    ret, frame = cap.read()
    if not ret:
        print("End of video stream")
        break

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    detected_faces = face_detector(gray_frame)

    for face in detected_faces:
        landmarks = landmark_predictor(gray_frame, face)
        landmarks = face_utils.shape_to_np(landmarks)

        left_eye_ear = calculate_ear(landmarks[36:42])
        right_eye_ear = calculate_ear(landmarks[42:48])

        left_eye_blink = get_blink_state(left_eye_ear, normal_ear)
        right_eye_blink = get_blink_state(right_eye_ear, normal_ear)

        if left_eye_blink == 0 or right_eye_blink == 0:
            sleep_counter += 1
            drowsy_counter = 0
            active_counter = 0
            if sleep_counter > 6:
                alert_status = "Sleeping!!!"
                status_color = (46, 29, 255)
        elif left_eye_blink == 1 or right_eye_blink == 1:
            sleep_counter = 0
            active_counter = 0
            drowsy_counter += 1
            if drowsy_counter > 6:
                alert_status = "Drowsy!!"
                status_color = (15, 193, 220)
        else:
            drowsy_counter = 0
            sleep_counter = 0
            active_counter += 1
            if active_counter > 6:
                alert_status = "Active"
                status_color = (86, 171, 22)
                alarm_play_count = 0  # Reset alarm counter when active

        # Display the alert status on the frame
        cv2.putText(frame, alert_status, (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.2, status_color, 3)
        
        # Play alarm sound if the user is sleepy or drowsy and the alarm hasn't played 30 times yet
        if (alert_status == "Sleeping!!!" or alert_status == "Drowsy!!") and alarm_play_count < 30:
            thread = threading.Thread(target=play_alarm_sound)
            thread.daemon = True
            thread.start()
            alarm_play_count += 1

        # Draw facial landmarks on the frame
        for (x, y) in landmarks:
            cv2.circle(frame, (x, y), 2, (255, 255, 255), -1)

    # Display the frame
    cv2.imshow("Driver Alert System", frame)

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) == ord('q'):
        break

# Release the video capture and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


Capturing initial frames to calculate normal EAR. Please keep your eyes open.
Normal EAR calculated: 0.3540697086690955
